- 结论
    - fp32（master copy） & fp16，两份权重
        - 4 bytes * number of parameters for fp32 training
        - 6 bytes * number of parameters for mixed precision training
        - FP16 格式的权重用于前向传播和反向传播
            - 激活值、梯度等中间结果都使用FP16存储,这些通常占据了训练过程中的大部分显存
        - FP32 格式的主权重用于参数更新（fp16 精度不够，容易把 weight 下溢为0）
        - 总的来说,FP16的中间结果节省的显存远超过额外存储FP32主权重所需的显存
    - 加快训练速度 & performance 未必会降级（也许会提升）
        - 减少显存使用,允许在相同硬件上训练更大的模型或使用更大的batch size
        - 减少显存使用,允许在相同硬件上训练更大的模型或使用更大的batch size
        - 保持准确性,通过在关键步骤中使用FP32
- 参考
    - https://towardsdatascience.com/understanding-mixed-precision-training-4b246679c7c4
    - https://sebastianraschka.com/blog/2023/llm-mixed-precision-copy.html

https://docs.nvidia.com/deeplearning/performance/mixed-precision-training/index.html

1. Maintain a primary copy of weights in FP32.
2. For each iteration:
    - Make an FP16 copy of the weights.
    - Forward propagation (FP16 weights and activations).
    - Multiply the resulting loss with the scaling factor S.
    - Backward propagation (FP16 weights, activations, and their gradients).
    - Multiply the weight gradient with 1/S.
    - Complete the weight update (including gradient clipping, etc.).